In [1]:
import torch
from models import unet
from utils import config

In [2]:
conf = config.ModelConfig({
    "audio_ctx_dim": 768
})

In [4]:
model = unet.UNetWithCrossAttention(conf).cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

loss_fn = torch.nn.MSELoss()

In [5]:
def get_time_embedding(timestep, dtype):
    freqs = torch.pow(10000, -torch.arange(start=0, end=160, dtype=dtype) / 160)
    x = torch.tensor([timestep], dtype=dtype)[:, None] * freqs[None]
    return torch.cat([torch.cos(x), torch.sin(x)], dim=-1)

In [6]:
BS = 1
image = torch.rand((BS, 3, 128, 128)).cuda()
sound = torch.rand((BS, 499, 768)).cuda()
time = get_time_embedding(0, torch.float).cuda()

In [7]:
optimizer.zero_grad()
loss = loss_fn(model(image, time), image)

In [8]:
loss.backward()
optimizer.step()

In [ ]:
def get_time_embedding(timestep, dtype=torch.float):
    assert len(timestep.shape) == 1, "timestep должен быть одномерным тензором размером [batch_size]"
    
    freqs = torch.pow(10000, -torch.arange(start=0, end=160, dtype=dtype) / 160)
    x = timestep[:, None].to(dtype) * freqs[None, :]  # [batch_size, 1] * [1, 160] = [batch_size, 160]
    return torch.cat([torch.cos(x), torch.sin(x)], dim=-1)  # [batch_size, 320]

In [ ]:
a = torch.tensor([1,2,3,4])

get_time_embedding(a).shape